In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from DATA.stock_invest_function import fetch_trade_data_multi_hscode

In [2]:
db_info = {
    'host': '192.168.0.230',
    'port': 3307,
    'user': 'stox7412',
    'password': 'Apt106503!~',
    'database': 'investar'
}

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{db_info['user']}:{db_info['password']}@{db_info['host']}:{db_info['port']}/{db_info['database']}"
)

# 테이블 이름
table_name = 'target_hs_code'

# 고유한 hs_code 값 추출 쿼리 실행
query = f"SELECT DISTINCT hs_code FROM {table_name}"
unique_hs_codes_df = pd.read_sql(query, con=engine)
hs_codes  = unique_hs_codes_df['hs_code'].unique().tolist()

indicator = 'expDlr'

df_real = fetch_trade_data_multi_hscode(db_info, hs_codes, indicator)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '192.168.0.230' (timed out)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# 분기 정보 추가
df_real['quarter'] = df_real['date'].dt.to_period('Q')

# 그룹별로 분기별 합산
df_quarterly = (
    df_real
    .groupby(['root_hs_code', 'quarter'])['value']
    .sum()
    .reset_index()
)

# 👉 분기 월말로 변환 (예: 2007Q1 → 2007-03-31)
df_quarterly['date'] = df_quarterly['quarter'].dt.to_timestamp(how='end')

# 👉 'quarter' 컬럼 제거
df_quarterly.drop(columns=['quarter'], inplace=True)

# 1단계: 문자열로 직접 변환하려면 to_datetime 이후에 바로 strftime
df_quarterly['date'] = pd.to_datetime(df_quarterly['date']).dt.strftime('%Y-%m-%d')

def create_yoy_growth_pivot(df_quarterly, start_date=None, end_date=None):
    """
    전년 동분기 대비 증가율을 pivot 형태로 변환하고 분석기간을 설정할 수 있는 함수

    Parameters:
    - df_quarterly (DataFrame): 'root_hs_code', 'date', 'yoy_growth' 포함된 데이터
    - start_date (str or None): 분석 시작일 (예: '2015-01-01')
    - end_date (str or None): 분석 종료일 (예: '2023-12-31')

    Returns:
    - pivot_df (DataFrame): 행: date, 열: root_hs_code, 값: yoy_growth
    """
    # Pivot
    pivot_df = df_quarterly.pivot(
        index='date',
        columns='root_hs_code',
        values='yoy_growth'
    ).sort_index()

    # inf 값 NaN 처리
    pivot_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # 분석 기간 슬라이싱 (날짜가 문자열이면 datetime으로 변환)
    pivot_df.index = pd.to_datetime(pivot_df.index)

    if start_date:
        pivot_df = pivot_df[pivot_df.index >= pd.to_datetime(start_date)]
    if end_date:
        pivot_df = pivot_df[pivot_df.index <= pd.to_datetime(end_date)]

    return pivot_df


# 전년 동분기 값 (4개 분기 전 값) 계산
df_quarterly['yoy_value'] = (
    df_quarterly
    .sort_values(['root_hs_code', 'date'])
    .groupby('root_hs_code')['value']
    .shift(4)
)

quarterly_pivot = create_yoy_growth_pivot(df_quarterly, start_date='2008-03', end_date='2025-03')

In [3]:
quarterly_pivot

NameError: name 'quarterly_pivot' is not defined